### Libraries

In [63]:
import os
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [49]:
DATA_DIR = r"D:\cell_images_small"
BATCH_SIZE = 32

### Data Transformation

In [50]:
def getTransforms():
    trainTransform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])
    
    evalTransform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    return trainTransform, evalTransform

### Spliting the dataset (70/15/15)

In [51]:
def splitDataset(dataset, random_state=42):
    baseDataset = datasets.ImageFolder(root=dataset,transform=None)

    indices = list(range(len(baseDataset)))
    targets = baseDataset.targets

    trainIdx, tempIdx = train_test_split(indices, test_size=0.3, random_state=random_state, stratify=targets)

    tempTargets = [targets[i] for i in tempIdx]
    valIdx, testIdx = train_test_split(tempIdx, test_size=0.5, random_state=random_state, stratify=tempTargets)

    return trainIdx, valIdx, testIdx

### Create Dataloaders Pipeline

In [52]:
def getDataloaders(dataset):
    trainTransform, evalTransform = getTransforms()
    trainIdx, valIdx, testIdx = splitDataset(dataset)

    trainFull = datasets.ImageFolder(dataset, transform=trainTransform)
    evalFull = datasets.ImageFolder(dataset, transform=evalTransform)

    trainDataset = Subset(trainFull, trainIdx)
    valDataset = Subset(evalFull, valIdx)
    testDataset = Subset(evalFull, testIdx)

    trainLoader = DataLoader(trainDataset, batch_size=BATCH_SIZE, shuffle=True) 
    valLoader = DataLoader(valDataset, batch_size=BATCH_SIZE, shuffle=False)
    testLoader = DataLoader(testDataset, batch_size=BATCH_SIZE, shuffle=False)

    return trainLoader, valLoader, testLoader

### Loading the Data

In [53]:
trainLoader, valLoader, testLoader = getDataloaders(DATA_DIR)

print("Train batches: ", len(trainLoader))
print("Validation batches: ", len(valLoader))
print("Test batches: ", len(testLoader))

Train batches:  5
Validation batches:  1
Test batches:  1


### Models

In [54]:
def get_model(model_name):
    
    if model_name == "resnet50":
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 2)
        
    elif model_name == "vgg16":
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(4096, 2)
        
    elif model_name == "vgg19":
        model = models.vgg19(pretrained=True)
        model.classifier[6] = nn.Linear(4096, 2)
        
    elif model_name == "efficientnet":
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
        
    elif model_name == "mobilenet":
        model = models.mobilenet_v2(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
        
    return model

### Training Function

In [55]:
def train_model(model, optimizer, criterion, epochs=5):
    
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        
        for images, labels in trainLoader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        val_acc = evaluate(model, valLoader)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = copy.deepcopy(model.state_dict())
        
        print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    model.load_state_dict(best_model)
    return model

### Evaluation Model

In [56]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return correct / total

### Hyperparameter Tuning

In [57]:
models_list = ["resnet50", "vgg16", "vgg19", "efficientnet", "mobilenet"]

learning_rates = [0.001, 0.0001]
optimizers_list = ["adam", "sgd"]

results = []

In [58]:
for model_name in models_list:
    for lr in learning_rates:
        for opt_name in optimizers_list:
            
            print(f"\nTraining {model_name} | LR={lr} | OPT={opt_name}")
            
            model = get_model(model_name)
            criterion = nn.CrossEntropyLoss()
            
            if opt_name == "adam":
                optimizer = optim.Adam(model.parameters(), lr=lr)
            else:
                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            
            trained_model = train_model(model, optimizer, criterion)
            test_acc = evaluate(trained_model, testLoader)
            
            results.append([model_name, lr, opt_name, test_acc])


Training resnet50 | LR=0.001 | OPT=adam


d:\MalariaDetection\myvenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\MalariaDetection\myvenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 4.9615, Val Acc: 0.5667
Epoch 2, Loss: 0.6713, Val Acc: 0.5000
Epoch 3, Loss: 0.3970, Val Acc: 0.6000
Epoch 4, Loss: 0.3843, Val Acc: 0.9667
Epoch 5, Loss: 0.4433, Val Acc: 0.6667

Training resnet50 | LR=0.001 | OPT=sgd
Epoch 1, Loss: 3.4473, Val Acc: 0.6000
Epoch 2, Loss: 2.7886, Val Acc: 0.8333
Epoch 3, Loss: 2.1777, Val Acc: 0.9000
Epoch 4, Loss: 1.4397, Val Acc: 0.9333
Epoch 5, Loss: 0.6616, Val Acc: 0.9333

Training resnet50 | LR=0.0001 | OPT=adam
Epoch 1, Loss: 1.9016, Val Acc: 0.9667
Epoch 2, Loss: 0.3854, Val Acc: 0.8667
Epoch 3, Loss: 0.2034, Val Acc: 0.7667
Epoch 4, Loss: 0.8702, Val Acc: 0.9667
Epoch 5, Loss: 0.1390, Val Acc: 1.0000

Training resnet50 | LR=0.0001 | OPT=sgd
Epoch 1, Loss: 3.5974, Val Acc: 0.5000
Epoch 2, Loss: 3.5645, Val Acc: 0.5333
Epoch 3, Loss: 3.3718, Val Acc: 0.6000
Epoch 4, Loss: 3.2488, Val Acc: 0.8000
Epoch 5, Loss: 3.1895, Val Acc: 0.8333

Training vgg16 | LR=0.001 | OPT=adam


d:\MalariaDetection\myvenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:52<00:00, 10.5MB/s] 


Epoch 1, Loss: 22.1681, Val Acc: 0.5000
Epoch 2, Loss: 3.7454, Val Acc: 0.5000
Epoch 3, Loss: 4.0226, Val Acc: 0.5000
Epoch 4, Loss: 3.5879, Val Acc: 0.5000
Epoch 5, Loss: 3.6866, Val Acc: 0.5000

Training vgg16 | LR=0.001 | OPT=sgd
Epoch 1, Loss: 3.7171, Val Acc: 0.6667
Epoch 2, Loss: 3.1025, Val Acc: 0.6000
Epoch 3, Loss: 2.4219, Val Acc: 0.8000
Epoch 4, Loss: 1.5869, Val Acc: 0.9000
Epoch 5, Loss: 1.3287, Val Acc: 0.9333

Training vgg16 | LR=0.0001 | OPT=adam
Epoch 1, Loss: 2.8908, Val Acc: 0.8000
Epoch 2, Loss: 2.0587, Val Acc: 0.9333
Epoch 3, Loss: 0.8343, Val Acc: 0.9667
Epoch 4, Loss: 0.5597, Val Acc: 0.9333
Epoch 5, Loss: 0.2532, Val Acc: 0.9667

Training vgg16 | LR=0.0001 | OPT=sgd
Epoch 1, Loss: 3.4637, Val Acc: 0.5000
Epoch 2, Loss: 3.4582, Val Acc: 0.5000
Epoch 3, Loss: 3.4555, Val Acc: 0.6333
Epoch 4, Loss: 3.3474, Val Acc: 0.7000
Epoch 5, Loss: 3.2857, Val Acc: 0.6667

Training vgg19 | LR=0.001 | OPT=adam


...

Epoch 1, Loss: 14.3468, Val Acc: 0.5000
Epoch 2, Loss: 4.3215, Val Acc: 0.5000
Epoch 3, Loss: 3.4600, Val Acc: 0.5000
Epoch 4, Loss: 3.8520, Val Acc: 0.5000
Epoch 5, Loss: 3.7573, Val Acc: 0.5000

Training vgg19 | LR=0.001 | OPT=sgd
Epoch 1, Loss: 3.7182, Val Acc: 0.5000
Epoch 2, Loss: 3.8335, Val Acc: 0.8667
Epoch 3, Loss: 2.5780, Val Acc: 0.8333
Epoch 4, Loss: 1.7690, Val Acc: 0.9333
Epoch 5, Loss: 0.7906, Val Acc: 0.9333

Training vgg19 | LR=0.0001 | OPT=adam
Epoch 1, Loss: 3.2204, Val Acc: 0.8333
Epoch 2, Loss: 1.0452, Val Acc: 0.9000
Epoch 3, Loss: 1.0598, Val Acc: 0.8667
Epoch 4, Loss: 0.4902, Val Acc: 0.9333
Epoch 5, Loss: 0.5359, Val Acc: 0.9000

Training vgg19 | LR=0.0001 | OPT=sgd
Epoch 1, Loss: 3.7824, Val Acc: 0.6667
Epoch 2, Loss: 3.7078, Val Acc: 0.6000
Epoch 3, Loss: 3.5287, Val Acc: 0.6333
Epoch 4, Loss: 3.6536, Val Acc: 0.7667
Epoch 5, Loss: 3.4165, Val Acc: 0.7333

Training efficientnet | LR=0.001 | OPT=adam


d:\MalariaDetection\myvenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:02<00:00, 8.78MB/s]


Epoch 1, Loss: 1.6657, Val Acc: 0.8667
Epoch 2, Loss: 0.5959, Val Acc: 0.9333
Epoch 3, Loss: 0.6369, Val Acc: 0.9667
Epoch 4, Loss: 0.9010, Val Acc: 0.9667
Epoch 5, Loss: 0.3691, Val Acc: 0.9667

Training efficientnet | LR=0.001 | OPT=sgd
Epoch 1, Loss: 3.3595, Val Acc: 0.6667
Epoch 2, Loss: 3.0294, Val Acc: 0.8333
Epoch 3, Loss: 2.5919, Val Acc: 0.9333
Epoch 4, Loss: 2.1272, Val Acc: 0.9667
Epoch 5, Loss: 1.7476, Val Acc: 0.9667

Training efficientnet | LR=0.0001 | OPT=adam
Epoch 1, Loss: 3.2341, Val Acc: 0.5000
Epoch 2, Loss: 2.1548, Val Acc: 0.5667
Epoch 3, Loss: 1.4815, Val Acc: 0.6667
Epoch 4, Loss: 1.0254, Val Acc: 0.9000
Epoch 5, Loss: 0.6317, Val Acc: 0.9667

Training efficientnet | LR=0.0001 | OPT=sgd
Epoch 1, Loss: 3.1616, Val Acc: 0.5667
Epoch 2, Loss: 3.0846, Val Acc: 0.6667
Epoch 3, Loss: 3.0321, Val Acc: 0.6667
Epoch 4, Loss: 3.0163, Val Acc: 0.7667
Epoch 5, Loss: 2.9610, Val Acc: 0.8000

Training mobilenet | LR=0.001 | OPT=adam


d:\MalariaDetection\myvenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 21.8MB/s]


Epoch 1, Loss: 0.9929, Val Acc: 0.9667
Epoch 2, Loss: 0.2286, Val Acc: 0.9667
Epoch 3, Loss: 1.6730, Val Acc: 0.9000
Epoch 4, Loss: 0.4449, Val Acc: 0.8667
Epoch 5, Loss: 0.1691, Val Acc: 0.8667

Training mobilenet | LR=0.001 | OPT=sgd
Epoch 1, Loss: 2.9815, Val Acc: 0.7000
Epoch 2, Loss: 2.1621, Val Acc: 0.5000
Epoch 3, Loss: 1.4361, Val Acc: 0.7333
Epoch 4, Loss: 0.8314, Val Acc: 0.8000
Epoch 5, Loss: 0.6416, Val Acc: 0.9667

Training mobilenet | LR=0.0001 | OPT=adam
Epoch 1, Loss: 2.6777, Val Acc: 0.5333
Epoch 2, Loss: 1.2801, Val Acc: 0.6000
Epoch 3, Loss: 0.6103, Val Acc: 0.8667
Epoch 4, Loss: 0.3230, Val Acc: 0.9333
Epoch 5, Loss: 0.2717, Val Acc: 0.9667

Training mobilenet | LR=0.0001 | OPT=sgd
Epoch 1, Loss: 3.6568, Val Acc: 0.3333
Epoch 2, Loss: 3.4172, Val Acc: 0.3667
Epoch 3, Loss: 3.3011, Val Acc: 0.5667
Epoch 4, Loss: 3.0181, Val Acc: 0.7000
Epoch 5, Loss: 2.8845, Val Acc: 0.7000


### Find the Best Model

In [71]:
import pandas as pd

results_df = pd.DataFrame(results, columns=["Model", "LR", "Optimizer", "Test Accuracy"])
results_df = results_df.sort_values("Test Accuracy", ascending=False).reset_index(drop=True)

results_df

,Model,LR,Optimizer,Test Accuracy
0,mobilenet,0.0001,adam,1.000000
1,efficientnet,0.0010,adam,1.000000
2,resnet50,0.0010,adam,0.966667
3,resnet50,0.0010,sgd,0.966667
4,mobilenet,0.0010,sgd,0.966667
5,mobilenet,0.0010,adam,0.966667
6,efficientnet,0.0001,adam,0.966667
7,vgg19,0.0001,adam,0.966667
8,vgg16,0.0010,sgd,0.966667
9,resnet50,0.0001,adam,0.966667


In [73]:
best_row = results_df.iloc[0]

best_model_name = best_row["Model"]
best_lr = best_row["LR"]
best_optimizer = best_row["Optimizer"]

print("Best Model:", best_model_name)
print("Learning Rate:", best_lr)
print("Optimizer:", best_optimizer)
print("Test Accuracy:", best_row["Test Accuracy"])

Best Model: mobilenet
Learning Rate: 0.0001
Optimizer: adam
Test Accuracy: 1.0


### Retrain the Best Model for final evaluation

In [74]:
model = get_model(best_model_name)
criterion = nn.CrossEntropyLoss()

if best_optimizer == "adam":
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
else:
    optimizer = optim.SGD(model.parameters(), lr=best_lr, momentum=0.9)

trained_best_model = train_model(model, optimizer, criterion)

d:\MalariaDetection\myvenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\MalariaDetection\myvenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 2.5091, Val Acc: 0.6000
Epoch 2, Loss: 0.9098, Val Acc: 0.7333
Epoch 3, Loss: 0.5753, Val Acc: 0.8000
Epoch 4, Loss: 0.3533, Val Acc: 0.9333
Epoch 5, Loss: 0.1452, Val Acc: 0.9667


### Final Evaluation Metrics

In [77]:
trained_best_model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in testLoader:
        outputs = trained_best_model(images)
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print("\n📊 Classification Report:\n")
print(classification_report(all_labels, all_preds))

cm = confusion_matrix(all_labels, all_preds)
print("\nConfusion Matrix:\n", cm)

roc = roc_auc_score(all_labels, all_preds)
print("\nROC-AUC Score:", roc)


📊 Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       0.94      1.00      0.97        15

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30


Confusion Matrix:
 [[14  1]
 [ 0 15]]

ROC-AUC Score: 0.9666666666666667
